[View in Colaboratory](https://colab.research.google.com/github/ale93111/Unet_dsb2018/blob/master/Unet_dsb2018.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [1]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
!pwd

/content


In [2]:
#!fusermount -u drive
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/kaggle/Unet_dsb2018")

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [3]:
#!pip install --upgrade pip
!pip install tqdm
!pip install keras
!pip install imgaug

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import os

# Root directory of the project
ROOT_DIR = os.getcwd()

print(ROOT_DIR)

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

#Dataset directory
dataset_path = os.path.join(ROOT_DIR, "RCNN_dataset_512_labels")

/content/drive/kaggle/Unet_dsb2018


In [5]:
import glob
import skimage.io
import numpy as np

#Find paths and load images and labels(=compressed masks)
img_paths = sorted(glob.glob(os.path.join(dataset_path,"*.png")))
msk_paths = sorted(glob.glob(os.path.join(dataset_path,"*.npy")))

print("Loading images...")
img_list = list(skimage.io.imread_collection(img_paths))
print("Loading labels...")
lab_list = [np.load(_path) for _path in msk_paths]

Loading images...
Loading labels...


In [6]:
from scipy.ndimage.morphology import binary_erosion
from tqdm import tqdm

def label_to_masks(labels):
    h, w = labels.shape
    n_msk = labels.max()
    masks = np.empty((h,w,n_msk),dtype=np.bool)
    for i in range(n_msk):
        masks[:,:,i] = labels==i+1
    return masks
  
def masks_to_label(msk):
    h, w, _ = msk.shape
    labels = np.zeros((h, w), dtype=np.uint16)
    for index in range(0, msk.shape[-1]):
        labels[msk[:,:,index] > 0] = index + 1
    return labels
  
def masks_to_gt(msk):
    h, w, _ = msk.shape
    gt = np.zeros((h, w), dtype=np.bool)
    for index in range(0, msk.shape[-1]):
        gt[msk[:,:,index] > 0] = True
    return gt[:,:,np.newaxis]
  

gt_list = []
for i,label in tqdm(enumerate(lab_list), total=len(lab_list)):
    #Convert to masks
    masks = label_to_masks(label)
    
    #Mask erosion as preprocessing
    for j in range(masks.shape[-1]):
        masks[:,:,j] = binary_erosion(masks[:,:,j].astype(np.uint8), border_value=1, iterations=1)
    masks = masks.astype(np.bool)
    
    #Get ground truths
    gt_list.append(masks_to_gt(masks))

100%|██████████| 809/809 [04:39<00:00,  2.89it/s]


In [7]:
import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.models as KM
import keras.utils as KU
import keras.losses as KLO

# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2, y_true)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0) #AGGIUNTO AXIS=0

def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    return 0.5 * KLO.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)
  
def Unet(img_size, GPU_COUNT=1):
    inputs = KL.Input((None, None, 3))
    s = KL.Lambda(lambda x: x / 255.0)(inputs)

    #TODO: make more general
    c1 = KL.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same')(s)
    n1 = KL.BatchNormalization(axis=3)(c1)
    a1 = KL.Activation("elu")(n1)
    c1 = KL.Dropout(0.2)(a1)
    c1 = KL.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
    n1 = KL.BatchNormalization(axis=3)(c1)
    a1 = KL.Activation("elu")(n1)
    p1 = KL.MaxPooling2D((2, 2))(a1)

    c2 = KL.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
    n2 = KL.BatchNormalization(axis=3)(c2)
    a2 = KL.Activation("elu")(n2)
    c2 = KL.Dropout(0.2)(a2)
    c2 = KL.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
    n2 = KL.BatchNormalization(axis=3)(c2)
    a2 = KL.Activation("elu")(n2)
    p2 = KL.MaxPooling2D((2, 2))(a2)

    c3 = KL.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
    n3 = KL.BatchNormalization(axis=3)(c3)
    a3 = KL.Activation("elu")(n3)
    c3 = KL.Dropout(0.3)(a3)
    c3 = KL.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
    n3 = KL.BatchNormalization(axis=3)(c3)
    a3 = KL.Activation("elu")(n3)
    p3 = KL.MaxPooling2D((2, 2))(a3)

    c4 = KL.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
    n4 = KL.BatchNormalization(axis=3)(c4)
    a4 = KL.Activation("elu")(n4)
    c4 = KL.Dropout(0.4)(a4)
    c4 = KL.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
    n4 = KL.BatchNormalization(axis=3)(c4)
    a4 = KL.Activation("elu")(n4)
    p4 = KL.MaxPooling2D(pool_size=(2, 2))(a4)

    c5 = KL.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
    n5 = KL.BatchNormalization(axis=3)(c5)
    a5 = KL.Activation("elu")(n5)
    c5 = KL.Dropout(0.4)(a5)
    c5 = KL.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
    n5 = KL.BatchNormalization(axis=3)(c5)
    a5 = KL.Activation("elu")(n5)

    u6 = KL.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(a5)
    u6 = KL.concatenate([u6, a4])
    c6 = KL.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u6)
    n6 = KL.BatchNormalization(axis=3)(c6)
    a6 = KL.Activation("elu")(n6)
    c6 = KL.Dropout(0.4)(a6)
    c6 = KL.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
    n6 = KL.BatchNormalization(axis=3)(c6)
    a6 = KL.Activation("elu")(n6)

    u7 = KL.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(a6)
    u7 = KL.concatenate([u7, a3])
    c7 = KL.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u7)
    n7 = KL.BatchNormalization(axis=3)(c7)
    a7 = KL.Activation("elu")(n7)
    c7 = KL.Dropout(0.4)(a7)
    c7 = KL.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
    n7 = KL.BatchNormalization(axis=3)(c7)
    a7 = KL.Activation("elu")(n7)

    u8 = KL.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(a7)
    u8 = KL.concatenate([u8, a2])
    c8 = KL.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u8)
    n8 = KL.BatchNormalization(axis=3)(c8)
    a8 = KL.Activation("elu")(n8)
    c8 = KL.Dropout(0.2)(a8)
    c8 = KL.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
    n8 = KL.BatchNormalization(axis=3)(c8)
    a8 = KL.Activation("elu")(n8)

    u9 = KL.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(a8)
    u9 = KL.concatenate([u9, a1], axis=3)
    c9 = KL.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same')(u9)
    n9 = KL.BatchNormalization(axis=3)(c9)
    a9 = KL.Activation("elu")(n9)
    c9 = KL.Dropout(0.2)(a9)
    c9 = KL.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same')(c9)
    n9 = KL.BatchNormalization(axis=3)(c9)
    a9 = KL.Activation("elu")(n9)

    outputs = KL.Conv2D(1, (1, 1), activation='sigmoid')(a9)
    
    # Add multi-GPU support.(not tested)
    if GPU_COUNT > 1:
        with tf.device("/cpu:0"):
            model = KM.Model(inputs=[inputs], outputs=[outputs])

        # make the model parallel
        model = KU.training_utils.multi_gpu_model(model, gpus=GPU_COUNT)

    else:
        model = KM.Model(inputs=[inputs], outputs=[outputs])

    return model

Using TensorFlow backend.


In [0]:
import imgaug
import random
from sklearn.model_selection import train_test_split

def random_crop(image, mask, crop_size = 256):
    h, w = image.shape[:2]
    y = random.randint(0, (h - crop_size))
    x = random.randint(0, (w - crop_size))
    img_crop = image[y:y + crop_size, x:x + crop_size]
    msk_crop = mask[y:y + crop_size, x:x + crop_size]
    return img_crop, msk_crop

def data_generator(img_list, msk_list, batch_size=2, crop_size=256, augmentation=None):
    
    batch_img = np.zeros((batch_size, crop_size, crop_size, 3))
    batch_msk = np.zeros((batch_size, crop_size, crop_size, 1))
    
    image_index = -1
    
    while True:
        for i in range(batch_size):
            image_index = (image_index + 1) % len(img_list)
            
            batch_img[i], batch_msk[i] = random_crop(img_list[image_index],
                                                     msk_list[image_index])
        
        if augmentation:
            aug_det = augmentation.to_deterministic()
            batch_img = aug_det.augment_images(batch_img)
            batch_msk = aug_det.augment_images(batch_msk)

        yield batch_img, batch_msk

In [0]:
batch_size = 16
crop_size = 256

augmentation = imgaug.augmenters.SomeOf((0, 2), [
        imgaug.augmenters.Fliplr(0.5),
        imgaug.augmenters.Flipud(0.5),
        imgaug.augmenters.OneOf([imgaug.augmenters.Affine(rotate=90),
                                 imgaug.augmenters.Affine(rotate=180),
                                 imgaug.augmenters.Affine(rotate=270)])
        #imgaug.augmenters.Multiply((0.8, 1.5)),
        #imgaug.augmenters.GaussianBlur(sigma=(0.0, 5.0))
])


img_train, img_val, gt_train, gt_val = train_test_split(img_list, gt_list, test_size=0.1, random_state=7, shuffle=True)

train_generator = data_generator(img_train, gt_train, batch_size=batch_size, crop_size=crop_size, augmentation=augmentation)
val_generator   = data_generator(img_val,   gt_val,   batch_size=batch_size, crop_size=crop_size, augmentation=augmentation)

In [13]:
import datetime
from keras.optimizers import Adam, SGD

NAME = 'bowl'
now = datetime.datetime.now()
LOG_DIR = os.path.join(MODEL_DIR, "{}{:%Y%m%dT%H%M}".format(NAME.lower(), now))
checkpoint_path = os.path.join(LOG_DIR, "U_net_{epoch:04d}.h5")

#Model
model = Unet(crop_size)

model.compile(optimizer=Adam(lr=0.001), loss=bce_dice_loss, metrics=[mean_iou,KLO.binary_crossentropy])

# Callbacks
callbacks = [keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=False),
             keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=0, save_weights_only=True)
]

model.fit_generator(train_generator, steps_per_epoch=len(img_train)/batch_size, epochs=5,
                    validation_data=val_generator, validation_steps=len(img_val)/batch_size, initial_epoch=0, callbacks=callbacks)

Epoch 1/5
46/45 [==============================] - 58s 1s/step - loss: -0.2438 - mean_iou: 0.3717 - binary_crossentropy: 0.4790 - val_loss: 4.1003 - val_mean_iou: 0.4276 - val_binary_crossentropy: 8.5873
Epoch 2/5
46/45 [==============================] - 47s 1s/step - loss: -0.5711 - mean_iou: 0.4295 - binary_crossentropy: 0.2136 - val_loss: 0.2785 - val_mean_iou: 0.4316 - val_binary_crossentropy: 1.2828
Epoch 3/5
46/45 [==============================] - 47s 1s/step - loss: -0.6951 - mean_iou: 0.4324 - binary_crossentropy: 0.1499 - val_loss: 0.5856 - val_mean_iou: 0.4322 - val_binary_crossentropy: 2.1122
Epoch 4/5
46/45 [==============================] - 47s 1s/step - loss: -0.7700 - mean_iou: 0.4313 - binary_crossentropy: 0.1177 - val_loss: -0.6759 - val_mean_iou: 0.4321 - val_binary_crossentropy: 0.2668
Epoch 5/5
19/45 [===========>..................] - ETA: 25s - loss: -0.8094 - mean_iou: 0.4325 - binary_crossentropy: 0.1127

46/45 [==============================] - 47s 1s/step - loss: -0.8074 - mean_iou: 0.4325 - binary_crossentropy: 0.1052 - val_loss: -0.6944 - val_mean_iou: 0.4323 - val_binary_crossentropy: 0.1966
